<a href="https://colab.research.google.com/github/MNourMoslem/Simple-Text-Classifier/blob/master/TextClassifier_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install The Libraries

In [ ]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is 

## Log To HuggingFace

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Import The Model, The Tokenizer and The IMDB Dataset


In [ ]:

id2label = {0 : "Negative", 1 : "Positive"}
label2id = {"Negative" : 0, "Positive" : 1}

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

checkpoint = "distilbert/distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2, id2label = id2label, label2id = label2id)

imdb = load_dataset("imdb", split = 'train+test')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Prepocess The Data

In [ ]:
imdb = imdb.train_test_split(test_size = 0.2)

def preprocess_data(example):
  return tokenizer(example['text'], truncation = True)

t_imdb = imdb.map(preprocess_data, batched = True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

## Import Accuracy Matrix

In [ ]:
import evaluate

accuracy = evaluate.load('accuracy')

In [ ]:
import numpy as np

def compute_metrics(result):
  preds, labels = result
  preds = np.argmax(preds, axis = 1)
  return accuracy.compute(predictions = preds, references = labels)

## Import Data Collator

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

## Set and Implement Trainer and TrainingArguments

In [ ]:
from transformers import Trainer, TrainingArguments

train_args = TrainingArguments(
  output_dir = "textclassifier-distilbert-imdb",
  overwrite_output_dir=True,
  learning_rate = 1e-4,
  num_train_epochs=2,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  weight_decay = 0.1,
  eval_strategy='epoch',
  save_strategy='epoch',
  load_best_model_at_end=True,
  push_to_hub = True
)

trainer = Trainer(
  model = model,
  args = train_args,
  tokenizer = tokenizer,
  train_dataset = t_imdb["train"],
  eval_dataset = t_imdb["test"],
  data_collator = data_collator,
  compute_metrics = compute_metrics
)

## Train The Model and Push to The Hub

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.216700,0.190629,0.927700
2,0.101300,0.241202,0.933700


TrainOutput(global_step=5000, training_loss=0.19267502212524415, metrics={'train_runtime': 3963.1065, 'train_samples_per_second': 20.186, 'train_steps_per_second': 1.262, 'total_flos': 1.0471696889580288e+16, 'train_loss': 0.19267502212524415, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/MNMoslem/textclassifier-distilbert-imdb/commit/7a5c792fbb4896fd525e8e33b7a39fad4f8a186c', commit_message='End of training', commit_description='', oid='7a5c792fbb4896fd525e8e33b7a39fad4f8a186c', pr_url=None, pr_revision=None, pr_num=None)

##Test The Result

In [ ]:
from transformers import pipeline

model_path = "MNMoslem/textclassifier-distilbert-imdb" # your own model path
pipe = pipeline('text-classification', model = model_path)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
text = ['I love this movie', 'i hate your movie']

pipe(text)

[{'label': 'Positive', 'score': 0.9968656897544861},
 {'label': 'Negative', 'score': 0.9947574138641357}]